In [61]:
import csv
import os
import networkit as nk
import datetime

In [62]:
graph_name = 'facebook100/Caltech36.mat'
#graph_name = 'email.graph'
initializations = ['trivial', 'editing', 'id_insert', 'ascending_degree_insert']
maxIterations = [0, 2, 5, 20]
sortPaths = [True, False]
randomness = [True, False]
plateauSize = [2, 5, 10]
seed = 0

In [63]:
def testrun (writer, G, init, m, s, r, p):
    if(init == 'trivial'):
        mover = nk.community.QuasiThresholdEditingLocalMover(G, [], m, s, r, [], p)
    if(init == 'editing'):
        G.indexEdges()
        editing = nk.community.QuasiThresholdEditingLinear(G)
        editing.run()
        mover = nk.community.QuasiThresholdEditingLocalMover(G, editing.getParents(), m, s, r, [], p)
    if(init == 'id_insert'):
        mover = nk.community.QuasiThresholdEditingLocalMover(G, [], m, s, r, [i for i in range(G.upperNodeIdBound())], p)        
    if(init == 'ascending_degree_insert'):
        degree_dict = {}
        for u in G.iterNodes():
            degree_dict[u] = G.degree(u)
        mover = nk.community.QuasiThresholdEditingLocalMover(G, [], m, s, r, [el[0] for el in sorted(degree_dict.items(), key=lambda kv: kv[1])], p)
        
    a = datetime.datetime.now()  
    mover.run()
    delta = datetime.datetime.now() - a
    row = [init]
    row.append(str(m))
    row.append(str(s))
    row.append(str(r))
    if(r):
        row.append(str(p))
    else:
        row.append('-')
    row.append(str(mover.getNumberOfEdits()))
    row.append(str(mover.getUsedIterations()))
    if(r):
        row.append(str(mover.getPlateauSize()))
    else:
        row.append('-')
    row.append(str(delta.total_seconds() * 1000))
    writer.writerow(row)




In [64]:
header = ['initialization', 'maxIterations', 'sortPaths', 'randomness', 'plateauSize', 'edits', 'usedIterations', 'actualPlateau', 'time']

base_string = ['#']
file_name = graph_name.split('.')[0].replace("/", "")
base_string.append(graph_name)
file_name += str(seed)
base_string.append('Seed:' + str(seed))
row_string = ['#']
s = 'Intializations:'
for init in initializations:
    file_name += init[0]
    s += init
    s+= '/'
row_string.append(s[:-1])
s = 'maxIterations:'
for el in maxIterations:
    file_name += str(el)
    s += str(el)
    s += '/'
row_string.append(s[:-1])
s = 'sortPaths:'
for el in sortPaths:
    if(el):
        file_name += '1'
    else:
        file_name += '0'
    s += str(el)
    s += '/'
row_string.append(s[:-1])
s = 'randomness:'
for el in randomness:
    if(el):
        file_name += '1'
    else:
        file_name += '0'
    s += str(el)
    s += '/'
row_string.append(s[:-1])
s = 'PlateauSize:'
for el in plateauSize:
    file_name += str(el)
    s += str(el)
    s += '/'
row_string.append(s[:-1])
file_name += '.csv'




output_path = "../output/QTM/" + graph_name.split('.')[0] + "/"
input_path = "../input/"
graph_path = input_path + graph_name
result_path = output_path + file_name
if not os.path.exists(output_path):
    os.makedirs(output_path)
result_file = open(result_path,"w+",newline='')
writer = csv.writer(result_file, delimiter=' ')
writer.writerow(base_string)
writer.writerow(row_string)
writer.writerow(header)

nk.setSeed(seed, False)

if(graph_name.split('/')[0] ==  "facebook100"):
    G = nk.graphio.readMat(input_path + graph_name, key="A")

if(graph_name.split('.')[1] == "graph"):
    G = nk.readGraph(graph_path, nk.Format.METIS)
if(graph_name.split('.')[1] == "edgelist"):
    G = nk.readGraph(graph_path, nk.Format.EdgeListTabOne)


for init in initializations:
    for m in maxIterations:
        for s in sortPaths:
            for r in randomness:
                if(r):
                    for p in plateauSize:
                        testrun(writer, G, init, m, s, r, p)
                else:
                    testrun(writer, G, init, m, s, r, 0)
                    
result_file.close()
                    





